In [ ]:
import os
gpu_device = 0    # número identificador del device puede ser: 0, 1, 2, o 3
os.environ["CUDA_VISIBLE_DEVICES"]=str(gpu_device)
from numba import cuda
cuda.select_device(0)

In [ ]:
import pandas as pd
import re
import random
import numpy as np
import math
import pickle
from sklearn.metrics import classification_report, accuracy_score
from datasets import Dataset,concatenate_datasets,load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
import torch

In [ ]:
%run ../utils/data.ipynb
%run ../utils/Generatives-small_data.ipynb

In [ ]:

# Cargar modelo y tokenizador
model_name = "microsoft/biogpt"
tokenizer = AutoTokenizer.from_pretrained(model_name)

inputs,targets,val_data,examples_raw = flatten_and_filter_dataset(ground_truth,report_data)  
inputs2,targets2,val_data2,examples_raw2= flatten_and_filter_dataset_no_prev(ground_truth,report_data) 

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-6,
    per_device_train_batch_size=4,
    num_train_epochs=2,
    weight_decay=0.01,
    save_total_limit=2,
)

training_args_all = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    # evaluation_strategy="epoch",
    learning_rate=1e-5,  # Increased for better adaptation
    per_device_train_batch_size=8,
    fp16=True,
    gradient_accumulation_steps=2,
    num_train_epochs=2,  # Increased epochs to account for repeated reports
    weight_decay=0.01,  # Prevents overfitting
    save_total_limit=2,
    eval_steps=40,
)
training_args_answer = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    # evaluation_strategy="epoch",
    learning_rate=1e-5,  # Lower for fine-tuning without losing generalization
    fp16=True,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    
    num_train_epochs=5,  # Shorter fine-tuning stage
    weight_decay=0.01,  # Lower weight decay to preserve learned features
    save_total_limit=2,
)

In [ ]:
dataset_final=pd.DataFrame.from_dict(inputs,orient='index')
targets=pd.DataFrame.from_dict(targets,orient='index')
val_data=pd.DataFrame.from_dict(val_data,orient='index')
dataset_final.columns=["text"]
targets.columns=["label"]
val_data.columns=["val_data"]
dataset_final=pd.concat([dataset_final, val_data],axis=1)

dataset_final2=pd.DataFrame.from_dict(inputs2,orient='index')
targets2=pd.DataFrame.from_dict(targets2,orient='index')
val_data2=pd.DataFrame.from_dict(val_data2,orient='index')
dataset_final2.columns=["text"]
targets2.columns=["label"]
val_data2.columns=["val_data"]
dataset_final2=pd.concat([dataset_final2, val_data2],axis=1)


In [ ]:
accuracies,output_dic=cross_validation_all(dataset_final,targets,training=True,low_beams=False,testing=False)

In [ ]:
for i,tipo in enumerate(questions):
    print(tipo, accuracies[i])

In [ ]:
model_name="biogpt"+"21e51e57"+"no_copy"+"description"+"16batch"+"final"
for tipo in questions:
    print(tipo)
    with open(f"results_dic_{tipo}/{model_name.split("/")[-1]}.pkl", "wb") as file:
        pickle.dump(output_dic[tipo], file)
   

In [ ]:
devices = cuda.gpus
dev=devices.current
cuda.select_device(dev.id)
cuda.close()